# Python Final Exam {.unnumbered}

## Problem 1.

![](images/Python_final1.png)

## Answer


In [ ]:
import pandas as pd
import numpy as np

def MVportfolio(asset, mu_p):
    # Calculate log returns (first column is date.)
    return_asset = np.log(asset.iloc[:, 1:]).diff()

    # Calculate expected log returns
    expect_asset = return_asset.mean()
    expect_asset_np = np.array(expect_asset)

    # Calculate covariance matrix
    cov_asset = return_asset.cov()
    cov_asset_np = np.array(cov_asset)

    # Use lagrange multipliers method for optimazation
    num_assets = return_asset.shape[1]
    u = np.ones(num_assets)

    a = np.block([
        [cov_asset_np, u[:, np.newaxis], expect_asset_np[:, np.newaxis]],
        [u[np.newaxis, :], np.zeros(1), np.zeros(1)],
        [expect_asset_np[np.newaxis, :], np.zeros(1), np.zeros(1)]
    ])

    b = np.concatenate([np.zeros(num_assets), [1], [mu_p]])

    lagrange = np.linalg.solve(a, b)
    
    # Optimal weights
    result = lagrange[:num_assets]

    # Portfolio variance
    Var_p = np.dot(result, np.dot(cov_asset, result))

    return result, Var_p

# Test
it = pd.read_csv('it.csv')
MVportfolio(asset = it, mu_p = -0.001)

In [ ]:
import matplotlib.pyplot as plot

# Range of mu_p
mu_p_values = np.arange(-0.001, 0.0011, 0.0001)

# Looping
portfolio_variances = []
for mu_p in mu_p_values:
    tmp = MVportfolio(asset=it, mu_p=mu_p)
    Var_p = tmp[1]
    portfolio_variances.append(Var_p)

# Plotting
plot.plot(portfolio_variances, mu_p_values)
plot.ylabel('Portfolio Variance')
plot.xlabel('Portfolio expected return')

## Problem 2.

![](images/Python_final2.png)

## Answer


In [ ]:
# 1. 'price.csv'의 'date'를 DatetimeIndex로 하는 pd.DataFrame price 생성
price = pd.read_csv('price.csv')
price['date'] = pd.to_datetime(price['date'])
price = price.set_index('date')
price

In [ ]:
# 2. price에서 2019년만 선택해서 price_sub로 저장
price_sub=price.loc['2019']
price_sub

In [ ]:
# 3. 열 별로 직전 대비 변동률을 구하고, 누적 곱하여 cum_ret 생성
returns = price_sub.pct_change()
cum_ret = (1 + returns).prod(axis=0) - 1 # 마지막에 1 빼주는게 맞음
cum_ret

In [ ]:
# 4. cum_ret에서 높은 순서로 10개 출력 및 가장 높은 종목의 시계열 그림 출력
top10_cum_ret = cum_ret.nlargest(10)
top10_cum_ret

In [ ]:
top_index = top10_cum_ret.index[0]
price_sub[top_index].plot()
plot.xlabel('Date')
plot.ylabel('Price')

In [ ]:
# 5. 열 별로 직전 대비 변동률의 표준편차를 계산하여 std 생성
std = returns.std(axis=0) * np.sqrt(252)
std

In [ ]:
# 6. std에서 0이거나 NaN인 경우 제외
std = std[(std != 0) & (~std.isna())]
std

In [ ]:
# 7. 종목 별 Sharpe ratio=cum_ret/std를 구하여 shrp 생성
shrp = cum_ret / std
shrp

In [ ]:
# 8. shrp에 NaN이 포함된 경우 shar의 최소값으로 대체
shrp.fillna(shrp.min(), inplace=True)
shrp

In [ ]:
# 9. shrp를 값이 큰 순서로 10개 종목 출력
top10_shrp = shrp.nlargest(10)
top10_shrp

In [ ]:
# 10. 위 10개 종목에 대한 cum_ret, std, shrp 값을 각 열로 하는 pd.DataFrame  final_result 생성
final_result = pd.DataFrame({
    'cum_ret': cum_ret[top10_shrp.index],
    'std': std[top10_shrp.index],
    'shrp': shrp[top10_shrp.index]
})
final_result

## Problem 3.

![](images/Python_final3.png)


In [ ]:
def ECallSimul_1(S0, K, T, r, sigma, M, I=250000): 
    import math 
    import random 
    S = [] 
    dt = T/M 
    for i in range(I): 
        path = [] 
        for t in range (M+1): 
            if t == 0 : 
                path.append(S0) 
            else: 
                z=random.gauss(0., 1.) 
                St = path[t-1] * math.exp( (r-0.5*sigma**2)*dt + sigma * math.sqrt(dt)*z ) 
                path.append(St) 
        S.append(path) 
    sum_val = 0.0 
    for path in S: 
        sum_val += max(path[-1]-K, 0) 
    C0 = math.exp(-r*T)*sum_val/I 
    return ( round(C0, 3) ) 

## Answer


In [ ]:
def ECallSimul_2(S0, K, T, r, sigma, M, I=250000):
    import numpy as np
    S = np.zeros((I,M+1))
    S[:,0] = S0
    dt = T/M
    z = np.random.standard_normal(size=(I,M))
    for t in range(1,M+1):
        S[:,t] = S[:,t-1]*np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z[:,t-1])
    sum_val = np.maximum(S[:,-1]-K, 0)
    C0 = np.exp(-r*T)*np.mean(sum_val)
    return ( round(C0, 3) )

In [ ]:
%time ECallSimul_1(S0=100.,K=105.,T=1.,r=0.05,sigma=0.2,M=50,I=250000)

In [ ]:
%time ECallSimul_2(S0=100.,K=105.,T=1.,r=0.05,sigma=0.2,M=50,I=250000)